In [ ]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import pickle
import warnings


from utils import *

from sensor_params import *
from models.unet1D_regressor import *
from models.training import *

# Dataset path 
DATABASE_PATH = r"C:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\Bases de datos\WARIFA\Mikael T1DM"

filename = "MIKAEL_data.json"

parent_directory = r"C:\Users\aralmeida\Downloads"
experiments_folder = r"\T1DM_pred_experiments" 

In [ ]:
# Load data from sgv_data. If executed, comment the line and load the data from the pickle file
# data , basal_data_dict, blood_pressure_dict, bolus_data_dict, exercise_data_dict, carbohydrates_data_dict, pump_events_data_dict, sgv_data_dict, sleep_data_dict, smgb_data_dict, steps_data_dict, weight_dict  = extract_Mikael_data(DATABASE_PATH, filename, ONLY_CGM = True)
with open(DATABASE_PATH+'\CGM.pk1', 'rb') as f:
    sgv_data_dict = pickle.load(f)
os.chdir(parent_directory)

In [ ]:
N = 48
tau = 1
PH = 5

In [ ]:
X, Y, X_times, Y_times = get_CGM_X_Y(sgv_data_dict, sensor_Mikael, N, tau, PH, experiments_folder, verbose = 0)

In [ ]:
os.getcwd()

In [ ]:
# # In case the dataset were already generated
# # Read npy files
# X = np.load('X.npy', allow_pickle=True)
# Y = np.load('Y.npy', allow_pickle=True)
# X_times = np.load('X_times.npy', allow_pickle=True)
# Y_times = np.load('Y_times.npy', allow_pickle=True)

In [ ]:
# Convert np.arrays to float32 to convert them to Tensorflow tensors
X = X.astype(np.float32)
Y = Y.astype(np.float32)

In [ ]:
# Dictionary to save the partitions 
training_partitions = {}

month_wise_4fold_cv(N, X, Y, X_times, training_partitions, 
                        shuffle=True, verbose=1)

In [ ]:
# Parameters for training
# Best parameters for the model
lr = 0.0001
batch_size = 2
epochs = 10

kernel_size = 3

In [ ]:
# Save only 5 first instances of X and Y
X_polla = training_partitions['month_wise_4fold_cv']['partitioned_dataset']['1-fold']["X_train"][:5]
Y_polla = training_partitions['month_wise_4fold_cv']['partitioned_dataset']['1-fold']["Y_train"][:5]

In [ ]:
# Train and save the models for the 4 folds
for fold in training_partitions['month_wise_4fold_cv']['partitioned_dataset']:
    train_model(X = X_polla,#training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["X_train"],
                Y = Y_polla,#training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Y_train"],
                N = N,
                tau = tau,
                kernel_size = kernel_size,
                predicted_points = sensor_Mikael["PREDICTED_POINTS"],
                sensor_name = sensor_Mikael,
                epochs = epochs,
                batch_size = batch_size,
                lr = lr,
                verbose = 1, 
                fold = fold)

In [ ]:
training_partitions['month_wise_4fold_cv']['partitioned_dataset']['1-fold'].keys()

In [ ]:
# Create 'evaluation' folder is it does not exist 
if not os.path.exists(os.getcwd()+r"\evaluation"):
    os.mkdir(os.getcwd()+r"\evaluation")

# Create keys in the dictionary for the test in the 4 folds if not already created
for fold in training_partitions['month_wise_4fold_cv']['partitioned_dataset']:
    if "Y_pred" not in training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold].keys(): 
        training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Y_pred"] = {}#np.empty(len(training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Y_test"]))
    if "RMSE" not in training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold].keys(): 
        training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["RMSE"] = {}
    if "ISO" not in training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold].keys(): 
        training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["ISO"] = {}
    if "Parker" not in training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold].keys(): 
        training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Parker"] = {}

In [ ]:
# Evaluate the models for the 4 folds
for fold in training_partitions['month_wise_4fold_cv']['partitioned_dataset']:

    # Declare variables aiming code readibility 
    X_test = training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["X_test"]
    Y_test = training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Y_test"]
    
    model = tf.keras.models.load_model(str(fold)+'.h5')
    Y_pred = model.predict(X_test)
    training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Y_pred"] = Y_pred  

    # Compute the RMSE
    training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["RMSE"] = np.sqrt(np.mean((Y_test - Y_pred)**2))

    # Mean and std of the RMSE
    rmse_mean = np.mean(training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["RMSE"])
    rmse_std = np.std(training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["RMSE"])
    print(str(fold)+ " Test RMSE:  ", rmse_mean, "±", rmse_std)

    # Go to the evaluation folder
    os.chdir(os.getcwd()+r"\evaluation")

    # Remove second dimension of Y_pred and Y_test to compute the metrics
    Y_pred = np.squeeze(Y_pred)
    Y_test = np.squeeze(Y_test)
    
    # Compute the metrics
    a, b= iso_percentage_metrics(Y_test, Y_pred)
    training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["ISO"] = a
    training_partitions['month_wise_4fold_cv']['partitioned_dataset'][fold]["Parker"] = b   

    a, b, c = bgISOAcceptableZone(ground_truth = Y_test, predictions = Y_pred, fold = fold, plot = True)
    a, b, c = parkes_EGA_chart(ground_truth = Y_test, predictions = Y_pred, fold = fold)
    
    # Go back to the parent folder
    os.chdir('..')

In [ ]:
# Remove second dimension of Y_pred and Y_test to compute the metrics
os.chdir('..')